#### Import libraries


In [4]:
import pandas as pd
import numpy as np

#### Load the it_hardware_sales_week.csv file

In [5]:
df = pd.read_csv('../data/inputs/raw/it_hardware_sales_week.csv')
df.head()

,Date,SalesAgent,Category,Amount,Payment Type
0,2025-05-04,Dana,Keyboard,1828.59,Cash
1,2025-05-01,Alice,Laptop,2247.92,Mobile Pay
2,2025-05-02,Eve,Printer,2223.36,Mobile Pay
3,2025-05-04,Eve,Laptop,1960.70,Mobile Pay
4,2025-04-30,Bob,Desktop,1622.98,Cash


Instead of always reading the top few rows of the dataset, we can look at a sample instead

In [16]:
df.sample(n=5, random_state=111)

,Date,SalesAgent,Category,Amount,Payment Type
15,2025-04-30,Alice,Monitor,1745.14,Mobile Pay
53,2025-05-02,Eve,Monitor,2139.87,Cash
44,2025-04-30,Chris,Printer,2354.12,Cash
16,2025-05-03,Alice,Laptop,1647.31,Mobile Pay
90,2025-04-29,Chris,Printer,1190.72,Debit Card


#### Let's look at the basic details of the data set

We can check the dimensions of the data set with ```shape```

In [10]:
df.shape

(100, 5)

So this data set has 100 rows and 5 columns

We'll use ```info()``` to get the data types and number of non-null values in a list of the features/columns

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          100 non-null    object 
 1   SalesAgent    100 non-null    object 
 2   Category      100 non-null    object 
 3   Amount        100 non-null    float64
 4   Payment Type  99 non-null     object 
dtypes: float64(1), object(4)
memory usage: 4.0+ KB


We can use ```describe()``` with the ```include='all'``` parameter to see the summary statistics for numberic columns, and the count, number of unique values, and most frequent value for categorical columns

In [8]:
df.describe(include='all')

,Date,SalesAgent,Category,Amount,Payment Type
count,100,100,100,100.000000,99
unique,7,5,7,NaN,4
top,2025-05-01,Alice,Laptop,NaN,Credit Card
freq,21,26,21,NaN,28
mean,NaN,NaN,NaN,1332.091400,NaN
std,NaN,NaN,NaN,765.484519,NaN
min,NaN,NaN,NaN,-1256.670000,NaN
25%,NaN,NaN,NaN,748.495000,NaN
50%,NaN,NaN,NaN,1455.030000,NaN
75%,NaN,NaN,NaN,1997.890000,NaN


We can check if any columns have null values like this :

In [9]:
df.isna().sum()

Date            0
SalesAgent      0
Category        0
Amount          0
Payment Type    1
dtype: int64

#### Looking things up

How can we look at individual rows/columns in the dataset?

In [15]:
df[['SalesAgent', 'Date']].head(3)

,SalesAgent,Date
0,Dana,2025-05-04
1,Alice,2025-05-01
2,Eve,2025-05-02


What if we want to look at a particular row?

In [17]:
df.loc[34]

Date            2025-04-30
SalesAgent             Bob
Category        Networking
Amount             1997.29
Payment Type          Cash
Name: 34, dtype: object

Or a particular row and column

In [18]:
df.loc[34, 'SalesAgent']

'Bob'

We can also use ```iloc``` to specify rows and columns by index, and also use a similar slicing syntax to python

In [22]:
df.iloc[10:20, 2:4]

,Category,Amount
10,Monitor,298.61
11,Laptop,1675.58
12,Mouse,62.40
13,Laptop,443.98
14,Laptop,1394.40
15,Monitor,1745.14
16,Laptop,1647.31
17,Desktop,599.46
18,Desktop,1794.84
19,Keyboard,631.26


We can select rows based on a condition

In [23]:
df[df['SalesAgent'] == 'Bob']

,Date,SalesAgent,Category,Amount,Payment Type
4,2025-04-30,Bob,Desktop,1622.98,Cash
6,2025-05-02,Bob,Keyboard,445.99,Mobile Pay
28,2025-05-03,Bob,Mouse,699.75,Mobile Pay
29,2025-05-02,Bob,Keyboard,647.77,Credit Card
34,2025-04-30,Bob,Networking,1997.29,Cash
47,2025-05-01,Bob,Laptop,956.89,Mobile Pay
50,2025-05-02,Bob,Monitor,1099.05,Mobile Pay
51,2025-04-30,Bob,Printer,2418.30,Cash
68,2025-05-01,Bob,Networking,2199.56,Mobile Pay
69,2025-05-04,Bob,Desktop,1864.88,Debit Card


In [ ]:
df[df['Amount'] > 1000]

,Date,SalesAgent,Category,Amount,Payment Type
37,2025-05-01,Chris,Laptop,-1256.67,Debit Card


We can also use ```isin()```

In [34]:
df_filtered = df[
    df['Category'].isin(['Desktop', 'Laptop']) &
    ~df['SalesAgent'].isin(['Chris', 'Bob'])
]
df_filtered

,Date,SalesAgent,Category,Amount,Payment Type
1,2025-05-01,Alice,Laptop,2247.92,Mobile Pay
3,2025-05-04,Eve,Laptop,1960.70,Mobile Pay
9,2025-04-30,Dana,Desktop,72.53,Mobile Pay
11,2025-04-30,Eve,Laptop,1675.58,Credit Card
13,2025-05-02,Eve,Laptop,443.98,Debit Card
14,2025-05-01,Eve,Laptop,1394.40,Debit Card
16,2025-05-03,Alice,Laptop,1647.31,Mobile Pay
17,2025-05-02,Alice,Desktop,599.46,Mobile Pay
18,2025-04-29,Alice,Desktop,1794.84,Cash
27,2025-04-29,Eve,Desktop,950.90,Debit Card


#### Grouping data

We can group data using the ```groupby()``` function

In [42]:
grouped = df.groupby(by=['Category', 'SalesAgent'])['Amount'].agg(['sum', 'mean', 'count', 'max'])
grouped

sum         mean  count      max
Category   SalesAgent                                       
Desktop    Alice        6827.87  1706.967500      4  2433.88
           Bob          4766.57  1588.856667      3  1864.88
           Chris        1631.41  1631.410000      1  1631.41
           Dana         2107.31  1053.655000      2  2034.78
           Eve          3831.85  1277.283333      3  1466.79
Keyboard   Alice        6167.44  1233.488000      5  2324.38
           Bob          1093.76   546.880000      2   647.77
           Chris        5504.60  1376.150000      4  2475.63
           Dana         4344.22  1086.055000      4  1828.59
           Eve          2013.35   671.116667      3   930.75
Laptop     Alice        6263.80  1565.950000      4  2247.92
           Bob          3022.26  1511.130000      2  2065.37
           Chris        3016.16   603.232000      5  2130.60
           Dana         5334.87  1066.974000      5  2032.93
           Eve          7232.35  1446.470000      5  1960.70
Monitor    Alice       10081.41  1440.201429      7  2387.13
           Bob          1099.05  1099.050000      1  1099.05
           Chris         361.30   361.300000      1   361.30
           Dana         4904.97  1226.242500      4  2135.28
           Eve          7393.51  1848.377500      4  2343.58
Mouse      Alice        2456.03   818.676667      3  1641.60
           Bob           699.75   699.750000      1   699.75
           Chris         737.89   737.890000      1   737.89
           Dana         2410.87  2410.870000      1  2410.87
           Eve          1497.54  1497.540000      1  1497.54
Networking Alice        3877.92  1938.960000      2  2235.51
           Bob          4196.85  2098.425000      2  2199.56
           Chris        3775.67  1258.556667      3  1878.90
           Dana         6525.87  1631.467500      4  2287.44
           Eve          3848.19  1924.095000      2  2291.42
Printer    Alice        2251.46  2251.460000      1  2251.46
           Bob          2418.30  2418.300000      1  2418.30
           Chris        7794.37  1299.061667      6  2354.12
           Dana         1013.09  1013.090000      1  1013.09
           Eve          2707.28  1353.640000      2  2223.36

Show the data for all agents in one category

In [41]:
grouped.loc['Laptop']

,sum,mean,count
SalesAgent,,,
Alice,6263.80,1565.950,4
Bob,3022.26,1511.130,2
Chris,3016.16,603.232,5
Dana,5334.87,1066.974,5
Eve,7232.35,1446.470,5


We can provide a custom aggregation function too

In [44]:
def get_range(x):
    return x.max() - x.min()

grouped = df.groupby(['Category', 'SalesAgent'])['Amount'].agg(
    total_sales='sum',
    average_sale='mean',
    range=get_range
)
grouped


total_sales  average_sale    range
Category   SalesAgent                                    
Desktop    Alice           6827.87   1706.967500  1834.42
           Bob             4766.57   1588.856667   586.17
           Chris           1631.41   1631.410000     0.00
           Dana            2107.31   1053.655000  1962.25
           Eve             3831.85   1277.283333   515.89
Keyboard   Alice           6167.44   1233.488000  2214.81
           Bob             1093.76    546.880000   201.78
           Chris           5504.60   1376.150000  1704.23
           Dana            4344.22   1086.055000  1540.51
           Eve             2013.35    671.116667   674.61
Laptop     Alice           6263.80   1565.950000  2109.86
           Bob             3022.26   1511.130000  1108.48
           Chris           3016.16    603.232000  3387.27
           Dana            5334.87   1066.974000  1752.67
           Eve             7232.35   1446.470000  1516.72
Monitor    Alice          10081.41   1440.201429  2262.40
           Bob             1099.05   1099.050000     0.00
           Chris            361.30    361.300000     0.00
           Dana            4904.97   1226.242500  1836.67
           Eve             7393.51   1848.377500   896.93
Mouse      Alice           2456.03    818.676667  1579.20
           Bob              699.75    699.750000     0.00
           Chris            737.89    737.890000     0.00
           Dana            2410.87   2410.870000     0.00
           Eve             1497.54   1497.540000     0.00
Networking Alice           3877.92   1938.960000   593.10
           Bob             4196.85   2098.425000   202.27
           Chris           3775.67   1258.556667  1737.40
           Dana            6525.87   1631.467500  1759.09
           Eve             3848.19   1924.095000   734.65
Printer    Alice           2251.46   2251.460000     0.00
           Bob             2418.30   2418.300000     0.00
           Chris           7794.37   1299.061667  2074.62
           Dana            1013.09   1013.090000     0.00
           Eve             2707.28   1353.640000  1739.44

#### Adding and changing columns :

We can create a new column - let's create a Day of Week column

First we need to convert the Date column into the correct type

In [47]:
df['Date'].dtype

dtype('O')

In [48]:
df['Datetime'] = pd.to_datetime(df['Date'], errors='raise')
df

,Date,SalesAgent,Category,Amount,Payment Type,Datetime
0,2025-05-04,Dana,Keyboard,1828.59,Cash,2025-05-04
1,2025-05-01,Alice,Laptop,2247.92,Mobile Pay,2025-05-01
2,2025-05-02,Eve,Printer,2223.36,Mobile Pay,2025-05-02
3,2025-05-04,Eve,Laptop,1960.70,Mobile Pay,2025-05-04
4,2025-04-30,Bob,Desktop,1622.98,Cash,2025-04-30
...,...,...,...,...,...,...
95,2025-05-03,Chris,Networking,141.50,Cash,2025-05-03
96,2025-05-01,Bob,Laptop,2065.37,Debit Card,2025-05-01
97,2025-05-03,Dana,Laptop,932.47,Credit Card,2025-05-03
98,2025-05-04,Chris,Monitor,361.30,Credit Card,2025-05-04


In [52]:
df['DayOfWeek'] = df['Datetime'].dt.day_name()
df

,Date,SalesAgent,Category,Amount,Payment Type,Datetime,DayOfWeek
0,2025-05-04,Dana,Keyboard,1828.59,Cash,2025-05-04,Sunday
1,2025-05-01,Alice,Laptop,2247.92,Mobile Pay,2025-05-01,Thursday
2,2025-05-02,Eve,Printer,2223.36,Mobile Pay,2025-05-02,Friday
3,2025-05-04,Eve,Laptop,1960.70,Mobile Pay,2025-05-04,Sunday
4,2025-04-30,Bob,Desktop,1622.98,Cash,2025-04-30,Wednesday
...,...,...,...,...,...,...,...
95,2025-05-03,Chris,Networking,141.50,Cash,2025-05-03,Saturday
96,2025-05-01,Bob,Laptop,2065.37,Debit Card,2025-05-01,Thursday
97,2025-05-03,Dana,Laptop,932.47,Credit Card,2025-05-03,Saturday
98,2025-05-04,Chris,Monitor,361.30,Credit Card,2025-05-04,Sunday
